In [93]:
# This will reload imports before executing code, allowing you to easily change contents of custom scripts
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append("../../src/features/")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import xgboost as xgb

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import model_selection, preprocessing

from sklearn.linear_model import LinearRegression

import seaborn as sns
import random
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, 20)]
random.shuffle(colors)

%env KMP_DUPLICATE_LIB_OK=TRUE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/c.christodoulou/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/c.christodoulou/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


env: KMP_DUPLICATE_LIB_OK=TRUE


In [83]:
# Specify the dir containing the data
DATA_DIR = "../../datasets/sberbank-russian-housing-market/"

files_data_dir = os.listdir(DATA_DIR)
data_list = [os.path.join(DATA_DIR,file) for file in files_data_dir]

data_list

['../../datasets/sberbank-russian-housing-market/macro.csv',
 '../../datasets/sberbank-russian-housing-market/.DS_Store',
 '../../datasets/sberbank-russian-housing-market/data_dictionary.txt',
 '../../datasets/sberbank-russian-housing-market/test.csv',
 '../../datasets/sberbank-russian-housing-market/train.csv',
 '../../datasets/sberbank-russian-housing-market/sample_submission.csv']

In [84]:
# Read the data
df = pd.read_csv("../../datasets/sberbank-russian-housing-market/train.csv")

In [85]:
df.shape

(30471, 292)

In [86]:
columns_to_drop = transformers.get_cols_to_drop(df, 0.1)

In [87]:
columns_to_drop_corr = transformers.get_corr_feature_drop(df, 0.95)

In [88]:
df = df.drop(columns_to_drop + columns_to_drop_corr, axis=1).copy()


In [90]:
df.shape

(30471, 151)

In [95]:
# Naive XGBoost - Use all columns, label-encode the catego
train_df = df.copy()

for f in train_df.columns:
    if train_df[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[f].values)) 
        train_df[f] = lbl.transform(list(train_df[f].values))

train, test, y_train, y_test = train_test_split(train_df.drop(["id", "timestamp", "price_doc"], axis=1), 
                                               train_df["price_doc"], random_state=10, test_size=0.25)

xgb_params = {
    'eta': 0.05,
    'max_depth': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    #'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
model = xgb.XGBRegressor(**xgb_params)
model.fit(train,y_train)
predictions = model.predict(test)
rmsle = np.sqrt(metrics.mean_squared_log_error(y_test, predictions))
print("RMSLE: %f" % (rmsle))

RMSLE: 0.481240
